In [1]:
import pandas as pd

dataframe = pd.read_csv('dataset.csv')
dataframe.head()

,Unnamed: 0,Name,Description,Importance
0,0,Yoga Session,Attend the 90-minute yoga session to improve f...,7
1,1,Update Budget,Review and update the monthly budget. Crucial ...,8
2,2,Watch Webinar,Attend a live webinar on digital marketing str...,4
3,3,Water Plants,Watering indoor plants to keep them alive. Imp...,7
4,4,Schedule Doctor’s Appointment,Scheduling an appointment for a routine health...,8


In [2]:
task_names = dataframe["Name"]
task_descriptions = dataframe["Description"]
importances = dataframe["Importance"]
print(len(task_names))

1000


In [3]:
concatenated = [task_names[i].lower() + " " + task_descriptions[i].lower() for i in range(len(task_names))]
print(concatenated[:2])

['yoga session attend the 90-minute yoga session to improve flexibility and reduce stress. important for well-being.', 'update budget review and update the monthly budget. crucial for financial health and planning.']


In [4]:
alphanumeric = []
for task in concatenated:
    new_task = ''.join([character if character.isalnum() else ' ' for character in task])
    alphanumeric.append(new_task)
print(alphanumeric[:2])

['yoga session attend the 90 minute yoga session to improve flexibility and reduce stress  important for well being ', 'update budget review and update the monthly budget  crucial for financial health and planning ']


In [5]:
tokens = []
for task in alphanumeric:
    tokens.append([word for word in task.split() if word != ''])
print(tokens[:5])

[['yoga', 'session', 'attend', 'the', '90', 'minute', 'yoga', 'session', 'to', 'improve', 'flexibility', 'and', 'reduce', 'stress', 'important', 'for', 'well', 'being'], ['update', 'budget', 'review', 'and', 'update', 'the', 'monthly', 'budget', 'crucial', 'for', 'financial', 'health', 'and', 'planning'], ['watch', 'webinar', 'attend', 'a', 'live', 'webinar', 'on', 'digital', 'marketing', 'strategies', 'useful', 'for', 'professional', 'development', 'but', 'it', 'will', 'be', 'recorded', 'for', 'later', 'viewing'], ['water', 'plants', 'watering', 'indoor', 'plants', 'to', 'keep', 'them', 'alive', 'important', 'for', 'plant', 'health', 'but', 'has', 'a', 'day', 'or', 'two', 'of', 'flexibility'], ['schedule', 'doctor', 's', 'appointment', 'scheduling', 'an', 'appointment', 'for', 'a', 'routine', 'health', 'check', 'up', 'important', 'for', 'monitoring', 'and', 'maintaining', 'health']]


In [6]:
vocabulary = set()
for task in tokens:
    for token in task:
        vocabulary.add(token)
vocabulary = list(vocabulary)
print(len(vocabulary))

1493


In [7]:
import numpy as np

word2index = dict()
for i, w in enumerate(vocabulary):
    word2index[w] = i
indices = [np.asarray([word2index[token] for token in task]) for task in tokens]
print(indices[:5])

[array([  43,  553,  733,  885,  919, 1208,   43,  553,  220, 1458,  887,
        266,  862, 1270, 1349,  690,  863, 1204]), array([1058, 1368,  165,  266, 1058,  885,  829, 1368,  246,  690, 1171,
        202,  266,  430]), array([1213, 1237,  733,  979,  272, 1237, 1295, 1174,  340,  539,  554,
        690,  281,  997,  193,  152,  846,  358,  647,  690,  242,  996]), array([ 352, 1462, 1461,  434, 1462,  220,  479, 1288, 1434, 1349,  690,
        665,  202,  193,  376,  979, 1444,  213,  903,  769,  887]), array([1044,  886,  191,   81,  902,  411,   81,  690,  979, 1492,  202,
        163,  273, 1349,  690,  565,  266, 1293,  202])]


In [10]:
from autograd import Tensor
from layers import Embedding, LSTMCell, MSELoss, Linear, Sequential
from optimisers import SGD

embedding = Embedding(vocab_size=len(vocabulary), dim=512)
model = Sequential([LSTMCell(n_inputs=512, n_hidden=512, n_outputs=1), LSTMCell(n_inputs=512, n_hidden=512, n_outputs=1)])
output_layer = Linear(n_inputs=512, n_outputs=1)

In [11]:
criterion = MSELoss()
optimiser = SGD(parameters=model.get_parameters() + embedding.get_parameters() + output_layer.get_parameters(), alpha=0.05)

In [12]:
X_train = indices[:int(len(indices) * 0.8)]
y_train = importances[:int(len(indices) * 0.8)]

X_test = indices[int(len(indices) * 0.8):]
y_test = importances[int(len(indices) * 0.8):]

print(type(y_train))

<class 'pandas.core.series.Series'>


In [13]:
min_loss = 1000
iterations = 10
for iteration in range(iterations):
    total_loss = 0
    
    hidden = model.init_hidden(batch_size=1)
    
    for task_i in range(len(X_train)):
        hidden = model.init_hidden(batch_size=1)
        
        for t in range(len(X_train[task_i])):
            input = Tensor([X_train[task_i][t]], autograd=True)
            lstm_input = embedding.forward(input=input)
            hidden = model.forward(input=lstm_input, hidden=hidden)
        
        output = output_layer.forward(hidden[0])
        target = Tensor(y_train[task_i], autograd=True)
        loss = criterion.forward(output, target)
        loss.backward()
        optimiser.step()
        
        total_loss += loss.data / len(X_train[task_i])
        epoch_loss = np.exp(total_loss / (task_i + 1))
        
        if epoch_loss < min_loss:
            min_loss = epoch_loss
            print()
        
        print(f"Iter: {iteration} - Alpha: {str(optimiser.alpha)[:5]} - Example {task_i + 1}/{len(X_train)} - Min Loss: {str(min_loss)[:5]} - Loss: {epoch_loss}", end='\r')
    optimiser.alpha *= 0.99

AttributeError: 'Sequential' object has no attribute 'init_hidden'

In [16]:
def predict(task_indices):
    hidden = model.init_hidden(batch_size=1)
    for t in range(len(task_indices)):
        input = Tensor([task_indices[t]], autograd=True)
        lstm_input = embedding.forward(input=input)
        hidden = model.forward(input=lstm_input, hidden=hidden)
    output = output_layer.forward(hidden[0])
    
    return output

In [17]:
for task_i in range(len(X_test)):
    output = predict(X_test[task_i])
    target = Tensor(np.array(y_test)[task_i], autograd=True)
    loss = criterion.forward(output, target)
    print(f"Task {task_i + 1}/{len(X_test)} - Loss: {loss.data}", end='\r')
    total_loss += loss.data / len(X_test[task_i])
loss_to_display = np.exp(total_loss / (task_i + 1))
print()
print(loss_to_display)

Task 200/200 - Loss: [6.42183281]]
[1.20739589]


In [18]:
import pickle as pkl

with open("importance_embedding.pkl", "wb") as file:
    pkl.dump(embedding, file)

with open("importance_model.pkl", "wb") as file:
    pkl.dump(model, file)

with open("importance_output.pkl", "wb") as file:
    pkl.dump(output_layer, file)

with open("word2index.pkl", "wb") as file:
    pkl.dump(word2index, file)